# Découverte  du capteur MCP9808

Ce cahier permet de découvrir le capteurMCP9808 qui est un capteur température de précision :


Figure du montage :
![Licence Charles-Henri Eyraud](images/Licence.jpg)

Photo du Montage :
![Licence Charles-Henri Eyraud](images/Licence.jpg)


## **Bloc1** : déclaration des bibliothèques de programmes et des pilotes de capteurs

La première ligne permet d'importer puis d'utiliser les bibliothèques de programmes qui sont utilisés pour interagir avec le capteur. MRAA fournit les programmes de bas niveau alors que UPM fournit les pilotes des capteurs. La seconde ligne  permet d'importer et de renommer la partie de la librairie dédiée au capteur MCP9808 en langage python

In [1]:
import _thread, mraa, upm, time

from upm import pyupm_mcp9808 as mcp

## **Bloc2** : déclaration des constantes statiques
 * adresses du bus et des capteurs utilisés; 
 * adresses de l'altitude du lieu 
 * paramètres de contrôle du capteur de précision 

Pour plus de précisions voir [site de description du pilote](https://iotdk.intel.com/docs/master/upm/classupm_1_1_m_c_p9808.html).

In [2]:
# Constant declarations
#----------------------
# i2c bus 1 on RaspBerry Pi
bus1 = 0
mcpAddr = 0x18

localAlt = 169 #Altitude de Lyon
UPPER_TEMP = 0x02
LOWER_TEMP = 0x03
CRIT_TEMP = 0x04


## **Bloc3** : Déclaration du capteur utilisé et choix de son nom : temp.

In [3]:
# create sensor on the bus 1
temp = mcp.MCP9808(bus1,mcpAddr)

## **Bloc4** : Acquisition et impression d'une mesure de température

In [4]:
tempHR = temp.getTemp ()
print ('Temp(MCP) : {0:.4f} °C'.format(tempHR))

Temp(MCP) : 26.6875 °C


## **Bloc5** : Déclaration des fonctions 

Lorsqu'on souhaite raliser des séries de mesures pour simplifier le code on utilise des fonctions.

Une fonction est un groupe de commandes  utilisées à plusieurs reprises et écrites une seule fois. On les utilise aussi pour structurer ou organiser la programmation.

In [11]:
# Sensor Functions
#---------

# MCP9808
#********
def get_mcpSensor(thisSensor):
    unity = thisSensor.isCelsius()
    upper = thisSensor.getMonitorReg (UPPER_TEMP)
    lower = thisSensor.getMonitorReg (LOWER_TEMP)
    critic = thisSensor.getMonitorReg (CRIT_TEMP)
    hyst = thisSensor.getHysteresis ()
    resol = thisSensor.getResolution ()
    idManuf = thisSensor.getManufacturer ()
    mcpId = thisSensor.getDevicedId ()
    if unity :
        print ('We are measuring in Celsius')
    else :
        print ('We are measuring in Fahrenheit')
    print ('\tLower T° : %.3f\n'%lower + '\tUpperT° : %.3f\n'%upper + '\tCritical T° : %.3f\n'%critic)
    print ('Hysteresis value : %.3f'%hyst)
    print ('Resolution : %.4f'%resol)
    print ('Manufacturer Id : %d'%idManuf)
    print ('Device Id : %d\n'%mcpId)
    return

def mes_mcpTemp(thisSensor):
    mcpTemp = thisSensor.getTemp ()
    return (mcpTemp)

def input_thread(CheckInput):
    check = input()
    CheckInput.append(check)
    return

def init_Thread () :
    CheckInput = []
    _thread.start_new_thread(input_thread, (CheckInput,))
    print ('Instructions :  \t"stop" pour arrêter les mesures.\n')
    print ('\t\t"resetBME" pour redémarrer le capteur BME280')
    return (CheckInput)
#Activate or test some sensors
#---------------------
#mcp.shutDown(False)


In [22]:
mes_mcpTemp(temp)

26.875

## **Bloc6** : Fonction principale comprenant un bloc de dialogue pour recevoir des commandes en ligne

In [18]:
# Main function
###############
def measureMeteoParams():
    nbmes = 0
    while True :
        option1 = input('Instructions :  \t"start" pour démarrer les mesures "check" pour la maintenance "exit" pour sortir.\n--> ')
        if option1 == 'exit' :
            break
        elif option1 == 'start' :
            break
        elif option1 == 'check' :
            option2 = 'ready'
            while option2.strip != exit :
                option2 = input('Taper "info" pour avoir des informations, "changer" pour modifier les paramètres des capteurs, "exit" pour sortir.\n---> ')
                if option2 == 'exit':
                    break
                elif option2 == 'info':
                    option3 = 4
                    while option3 != 0 :
                        option3 = input('Informations sur les capteurs : MCP9808 (1) exit (0)\n----> ')
                        if option3 == '0' :
                            break
                        elif option3 == '1' :
                            get_mcpSensor(temp)
                        else :
                            print ('Code inconnu : %s'%option3)
                elif option2 == 'changer' :
                    option3 = 3
                    while option3 != 0 :
                        option3 = input('Changement des paramètres des capteurs :\n\tMCP9808-precision (1)\n\tMCP9808-hysteresis (2)\n\tBME280_mode (3)\n\t sortir (0)\n----> ')
                        if option3 == '0' :
                            break
                        elif option3 == '1' :
                            val = input("precision value (LOW MED HI MAX) : ")
                            if val == 'LOW' :
                                temp.setResolution (0x00)
                            elif val == 'MED' :
                                temp.setResolution (0x01)
                            elif val == 'HI' :
                                temp.setResolution (0x02)
                            elif val == 'MAX' :
                                temp.setResolution (0x03)
                            else :
                                print ('Commande inconnue : "%s"\n'%val)
                        elif option3 == '2' :
                            val = input("Hysteresis value  (HYST_0 HYST_1_5 HYST_3_0 HYST_6_0) : ")
                            if val == 'HYST_0' :
                                temp.setHysteresis (0x0000)
                            elif val == 'HYST_1_5' :
                                temp.setHysteresis (0x0002)
                            elif val == 'HYST_3_0' :
                                temp.setHysteresis (0x0004)
                            elif val == 'HYST_6_0' :
                                temp.setHysteresis (0x0006)
                            else :
                                print ('Commande inconnue : "%s"\n'%val)
                        else :
                            print ('Code inconnu : "%s"\n'%option3)
                else :
                        print ('Commande inconnue : "%s"\n'%option2)
        else :
            print ('Commande inconnue : "%s"\n'%option1)
# Main loop
    if option1.strip() == 'start' :
        CheckInput = init_Thread()
        while True :
            if CheckInput == ['stop']: 
                print ('Arrêt demandé.')
                break
            elif CheckInput != [] :
                print ('Commande inconnue : "%s" \nReprise des mesures\n'%CheckInput)
                CheckInput = []
                _thread.start_new_thread(input_thread, (CheckInput,))
                nbmes = 0
            else :
                time.sleep (0.5)
            tempHR = mes_mcpTemp(temp)
            if nbmes == 0 :
                print ('Temp(MCP) : ' )
                nbmes += 1
            elif nbmes == 9 :
                nbmes = 0
            else :
                nbmes += 1
            print ('{0:.4f} °C'.format(tempHR))
            time.sleep (5)
    else :
        print ('Bye...')
    return
    

measureMeteoParams()


Instructions :  	"start" pour démarrer les mesures "check" pour la maintenance "exit" pour sortir.
--> start
Instructions :  	"stop" pour arrêter les mesures.

		"resetBME" pour redémarrer le capteur BME280
Temp(MCP) : 
26.6250 °C
t
Commande inconnue : "['t']" 
Reprise des mesures

Temp(MCP) : 
26.6250 °C
stop
Arrêt demandé.
